In [25]:
!sudo apt-get install libportaudio2

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  libportaudio2
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 65.3 kB of archives.
After this operation, 223 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Fetched 65.3 kB in 1s (55.3 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 121925 fi

In [26]:
!sudo apt-get install portaudio19-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libportaudiocpp0
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudiocpp0 portaudio19-dev
0 upgraded, 2 newly installed, 0 to remove and 45 not upgraded.
Need to get 122 kB of archives.
After this operation, 703 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 122 kB in 1s (88.0 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
de

In [27]:
pip install pyaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 1.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyaudio: filename=PyAudio-0.2.14-cp310-cp310-linux_x86_64.whl size=63860 sha256=2acb600a3b36aa5ddc8422b5aa52c85b294b5be48336a842c6546470ba81ae86
  Stored in directory: /root/.cache/pip/wheels/d6/21/f4/0b51d41ba79e51b16295cbb096ec49f334792814d545b508c5
Successfully built pyaudio


In [28]:
pip install --upgrade git+https://github.com/huggingface/transformers.git accelerate datasets[audio]

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-328ft152
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-328ft152
  Resolved https://github.com/huggingface/transformers.git to commit 048f599f3506e57e0a595b455d9d2834c8d45023
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 5.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvid

In [1]:
import os
import torchaudio
import torch
import re
import string
import inflect
from jiwer import cer
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import load_metric

In [2]:
# Check device
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

# Load model and processor
model_id = "openai/whisper-large-v3"
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)
processor = AutoProcessor.from_pretrained(model_id)

# Create the pipeline
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=256,
    chunk_length_s=20,
    stride_length_s=5,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
# Set paths to the datasets
dataset_dir = os.path.expanduser("/content/drive/MyDrive")
audio_dir = os.path.join(dataset_dir, "wav")
corrected_txt_dir = os.path.join(dataset_dir, "corrected_txt")

In [4]:

# Define functions for preprocessing and text normalization
def remove_punctuation_from_text_regex(text):
    pattern = r'[{}]'.format(re.escape(string.punctuation))
    text_no_punct = re.sub(pattern, '', text)
    return text_no_punct.upper()

def convert_numbers_to_words(text):
    p = inflect.engine()
    def replace_func(match):
        num = match.group()
        return p.number_to_words(num)
    converted_text = re.sub(r'\b\d+\b', replace_func, text)
    return converted_text

# Audio preprocessing functions
def normalize_audio(audio_path):
    audio = torchaudio.load(audio_path)[0]
    return torchaudio.transforms.Resample(orig_freq=audio.shape[0], new_freq=16000)(audio)

# Speech file processing
def speech_file_to_array_fn(file_path):
    speech_array, sampling_rate = torchaudio.load(file_path)
    resampler = torchaudio.transforms.Resample(sampling_rate, 16000)
    speech = resampler(speech_array).squeeze().numpy()
    return speech

def predict(speech):
    predicted_text = pipe(speech, generate_kwargs={"language": "english"})
    return predicted_text

# Read transcript
def read_transcript(file_path):
    with open(file_path, 'r') as f:
        return f.read().strip()



# Function to fine-tune model
def fine_tune_model(model, train_dataset, eval_dataset):
    training_args = Seq2SeqTrainingArguments(
        output_dir="./whisper-finetuned",
        per_device_train_batch_size=4,
        num_train_epochs=3,
        predict_with_generate=True,
        fp16=True,
        save_steps=1000,
        logging_steps=100,
        eval_steps=500,
        save_total_limit=2,
        evaluation_strategy="steps",
        learning_rate=2e-5,
        weight_decay=0.01,
    )

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=processor.tokenizer,
    )

    trainer.train()



In [5]:
# Process dataset
def process_dataset(audio_dir, transcript_dir):
    audio_files = [os.path.join(audio_dir, file) for file in os.listdir(audio_dir) if file.endswith(".wav")]
    audio_files.sort()
    predictions = []
    references = []

    for audio_file in audio_files:
        transcript_file_name = os.path.splitext(os.path.basename(audio_file))[0] + ".txt"
        transcript_path = os.path.join(transcript_dir, transcript_file_name)

        if not os.path.exists(transcript_path):
            print(f"Transcript file not found for audio: {audio_file}")
            continue

        # Read ground truth transcription
        reference_text = read_transcript(transcript_path)
        reference_text = remove_punctuation_from_text_regex(reference_text)

        if not reference_text:
            print(f"Empty reference text found for audio: {audio_file}")
            continue

        references.append(reference_text)

        # Transcribe the audio file
        speech = speech_file_to_array_fn(audio_file)
        predicted_text = predict(speech)
        predicted_text = convert_numbers_to_words(predicted_text['text'])
        predicted_text = remove_punctuation_from_text_regex(predicted_text)
        predictions.append(predicted_text)

        print(f"Transcript for {audio_file}: {reference_text}")
        print(f"Prediction for {audio_file}: {predicted_text}")

    return predictions, references

In [6]:
# Function to calculate CER
def calculate_cer(predictions, references):
    cer_metric = load_metric("cer")
    cer_value = cer_metric.compute(predictions=predictions, references=references)
    return cer_value

# Process dataset and compute CER
predictions_corrected, references_corrected = process_dataset(audio_dir, corrected_txt_dir)
cer_corrected = calculate_cer(predictions_corrected, references_corrected)

print(f"CER for corrected transcripts: {cer_corrected:.4f}")

# Placeholder for fine-tuning call
# train_dataset and eval_dataset need to be defined as per your data
# fine_tune_model(model, train_dataset, eval_dataset)

/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:480: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
You have passed language=english, but also have set `forced_decoder_ids` to [[1, None], [2, 50360]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of language=english.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Transcript for /content/drive/MyDrive/wav/0000003b8fd9bc22877135b42b04c49d4860312b001be688723ecc5d.wav: IN THIS PARTICULAR SESSION WE WILL BE DISCUSSING HOW TO BENCHMARK SOMETHING THAT I GOT TO
Prediction for /content/drive/MyDrive/wav/0000003b8fd9bc22877135b42b04c49d4860312b001be688723ecc5d.wav:  IN THIS PARTICULAR SESSION WE WILL BE DISCUSSING HOW TO BENCHMARK SOMETHING THAT I HAVE GOT
Transcript for /content/drive/MyDrive/wav/00000682f31904acc560fa359512e7bdd487b11efe36145a56874e30.wav: ELECTRIC FIELD LIKE THIS WHY BECAUSE FIRST OF ALL THIS ELECTRIC FIELD INTERN CREATES
Prediction for /content/drive/MyDrive/wav/00000682f31904acc560fa359512e7bdd487b11efe36145a56874e30.wav:  ELECTRIC FIELD LIKE THIS WHY BECAUSE FIRST OF ALL THIS ELECTRIC FIELD IN TURN CREATES
Transcript for /content/drive/MyDrive/wav/0000068bfcd8e252fd9ec8225bd1fdb47378a009f8afa00d4e998df5.wav: TWO Z INVARIANT MAP AH J TILDE SO THAT THE FOLLOWING DIAGRAMS
Prediction for /content/drive/MyDrive/wav/0000068bfcd8e252fd9ec

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Transcript for /content/drive/MyDrive/wav/00001e691520008b2f975e9ca29822bb6d19fca95f0cb5df094a7bc6.wav: R SO THIS IS THE FLUX THAT IS LINKING OUTSIDE THE WIRE
Prediction for /content/drive/MyDrive/wav/00001e691520008b2f975e9ca29822bb6d19fca95f0cb5df094a7bc6.wav:  SO THIS IS THE FLUX THAT IS LINKING OUTSIDE THE WIRE
Transcript for /content/drive/MyDrive/wav/000020b0688f8ab2d012242e10acc10e0b4aaf6882b075d66848fe36.wav: FORCES THAT ARE BEEN GOING TO BE GENERATED WHILE EMISSIONING THIS PARTICULAR OR THAT
Prediction for /content/drive/MyDrive/wav/000020b0688f8ab2d012242e10acc10e0b4aaf6882b075d66848fe36.wav:  PROCESSES THAT ARE BEING GOING TO BE GENERATED WHILE MACHINING THIS PARTICULAR OR THAT
Transcript for /content/drive/MyDrive/wav/000020db7e99a1de454a92346534c843f7f712fd7d3160a7d575af4a.wav: SAY AT THIS POINT THIS PLUS THIS PLUS THIS AND SO ON WILL GIVE ME AT THAT CORRESPONDING
Prediction for /content/drive/MyDrive/wav/000020db7e99a1de454a92346534c843f7f712fd7d3160a7d575af4a.wav:  SAY A

<ipython-input-6-fe23b8f533cb>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  cer_metric = load_metric("cer")


The repository for cer contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/cer.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y
CER for corrected transcripts: 0.1039
